In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [4]:
# Generate sample dataset
def generate_sample_dataset(n=1000):
    np.random.seed(42)
    data = {
        "Soil_Moisture": np.random.uniform(20, 90, n),
        "Air_Temperature": np.random.uniform(20, 28, n),
        "Air_Humidity": np.random.uniform(40, 45, n),
        "Light_Intensity": np.random.uniform(500, 30000, n)
    }
    df = pd.DataFrame(data)
    df["Water_Next"] = df["Soil_Moisture"].apply(lambda x: 1 if x < 40 else 0)
    df["Sunlight_Hours"] = df["Light_Intensity"].apply(lambda x: 3 if x < 1500 else 0)
    df.to_csv("sample_dataset1.csv", index=False)
    return df

In [5]:
# Load dataset from path
def load_dataset(path):
    df = pd.read_csv(path)
    return df

# Generate and save sample dataset
generate_sample_dataset(300)

,Soil_Moisture,Air_Temperature,Air_Humidity,Light_Intensity,Water_Next,Sunlight_Hours
0,46.217808,20.413454,40.844675,6632.644527,0,0
1,86.550001,24.250837,41.392952,1282.700014,0,3
2,71.239576,24.325081,40.885052,5852.345335,0,0
3,61.906094,25.099439,40.443513,17699.726049,0,0
4,30.921305,25.808731,40.603179,12932.024242,1,0
...,...,...,...,...,...,...
295,56.557028,23.791693,41.024921,26210.335781,0,0
296,73.899549,25.340462,41.465739,29217.924591,0,0
297,35.107472,21.378559,44.481679,29081.896731,1,0
298,63.602333,21.538312,40.065010,22614.729036,0,0


In [6]:
df = load_dataset('sample_dataset1.csv')

# Handle missing values
df = df.dropna()

In [7]:
# Handle outliers using IQR
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df[~((df < lower_bound) | (df > upper_bound)).any(axis=1)]


In [8]:
# Features and labels
X = df[["Soil_Moisture", "Air_Temperature", "Air_Humidity", "Light_Intensity"]]
y1 = df["Water_Next"]  # When to water next
y2 = df["Sunlight_Hours"]  # Hours to keep in sunlight

In [ ]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
# Splitting data
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)
# Train Decision Tree Classifier for watering
model_watering = DecisionTreeClassifier()
model_watering.fit(X_train, y1_train)


DecisionTreeClassifier()

In [11]:
# Train Decision Tree Regressor for sunlight hours
model_sunlight = DecisionTreeClassifier()
model_sunlight.fit(X_train, y2_train)


DecisionTreeClassifier()

In [ ]:
# Predict and evaluate
predictions_watering = model_watering.predict(X_test)
predictions_sunlight = model_sunlight.predict(X_test)

In [13]:
print("Watering Prediction Accuracy:", accuracy_score(y1_test, predictions_watering))
print("Sunlight Hours Prediction Accuracy:", accuracy_score(y2_test, predictions_sunlight))

Watering Prediction Accuracy: 0.9827586206896551
Sunlight Hours Prediction Accuracy: 1.0


In [14]:
# Train another model for sunlight adjustment
y2 = df["Sunlight_Hours"]  # Whether to increase sunlight
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)
model_sunlight = DecisionTreeClassifier()
model_sunlight.fit(X_train, y2_train)

predictions_sunlight = model_sunlight.predict(X_test)
print("Sunlight Adjustment Accuracy:", accuracy_score(y2_test, predictions_sunlight))


Sunlight Adjustment Accuracy: 1.0


In [15]:
# Function to make new predictions
def predict_watering_and_sunlight(soil_moisture, temp, humidity, light_intensity):
    input_data = scaler.transform(np.array([[soil_moisture, temp, humidity, light_intensity]]))
    water_next = model_watering.predict(input_data)[0]
    sunlight_hours = model_sunlight.predict(input_data)[0]
    
    return {
        "Water Next": "Yes" if water_next == 1 else "No",
        "Sunlight Hours": sunlight_hours
    }

In [16]:
# Example Prediction
sample_prediction = predict_watering_and_sunlight(30, 25, 60, 3000)
print(sample_prediction)


{'Water Next': 'Yes', 'Sunlight Hours': 0}


c:\Users\dhyan\anaconda\envs\crime_analysis_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [17]:
import joblib

# Assuming you have already trained your models and scaler
# Save the scaler
joblib.dump(scaler, 'sensor_scaler.save')

# Save the watering model
joblib.dump(model_watering, 'watering_model.pkl')

# Save the sunlight model
joblib.dump(model_sunlight, 'sunlight_model.pkl')

print("Models and scaler saved successfully!")


Models and scaler saved successfully!
